In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 작업 폴더 생성 및 이동 (VUDENC 프로젝트를 저장할 위치)
# 경로에 본인의 구글 드라이브 ID나 원하는 폴더명을 넣으셔도 됩니다.
import os
workspace_path = '/content/drive/MyDrive/fortmp/w2v'
os.makedirs(workspace_path, exist_ok=True)
%cd {workspace_path}

print(f"현재 작업 위치: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/fortmp/w2v
현재 작업 위치: /content/drive/MyDrive/fortmp/w2v


In [ ]:
# 1. VUDENC GitHub 저장소 클론 (이미 있다면 실행 X)
if not os.path.exists('VulnerabilityDetection'):
    !git clone https://github.com/LauraWartschinski/VulnerabilityDetection.git
else:
    print("VulnerabilityDetection 폴더가 이미 존재합니다.")

# 2. 프로젝트 폴더로 이동
%cd VulnerabilityDetection

# 3. 필요 라이브러리 설치
!pip install gensim

VulnerabilityDetection 폴더가 이미 존재합니다.
/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection


In [ ]:
!pwd

/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection


In [ ]:
# 1. 훈련 스크립트가 있는 폴더로 이동
%cd Code

# 2. Zenodo에서 토큰화된 코퍼스 zip 파일 다운로드
# (pythontraining_withString_X.py.zip 파일, 약 1.4GB)
# !wget "https://zenodo.org/record/3732235/files/pythontraining_withString_X.py.zip?download=1" -O pythontraining_withString_X.py.zip

# 로컬에 있는 압축 파일 경로
local_rar_path = '/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection/Code/w2v/pythontraining_withString.rar'
output_dir = '/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection/Code/w2v' # 압축 해제할 경로

# 3. 다운로드한 zip 파일 압축 해제 (rar 파일이므로 unrar 사용)
# unrar 명령어가 설치되어 있지 않다면 설치 필요: !apt-get install unrar
# 압축 해제 전, unrar 설치 여부 확인 및 설치
!apt-get update && apt-get install -y unrar

# 압축 해제 실행 (대상 경로 지정 -t 옵션)
# -x 옵션은 경로를 유지하면서 압축을 해제합니다.
!unrar x {local_rar_path} {output_dir}

print("\n토큰화된 코퍼스 준비 완료!")
!ls -lh {output_dir} # 파일 목록과 크기 확인

/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection/Code
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Fetched 257 kB in 22s (11.5 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.s

In [ ]:
!pwd

/content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection/Code


In [ ]:
import nltk
try:
    nltk.data.find('tokenizers/punkt_tab/english/')
except LookupError:
    nltk.download('punkt_tab')
    print("NLTK 'punkt_tab' 리소스 다운로드 완료.")

In [ ]:
import nltk
from gensim.models import Word2Vec, KeyedVectors
import os.path
import pickle
import sys

all_words = []

mode = "withString" #default



# Loading the training corpus
print("Loading " + mode)
with open('w2v/pythontraining' + '_'+mode+"_X", 'r') as file:
    pythondata = file.read().lower().replace('\n', ' ')

print("Length of the training file: " + str(len(pythondata)) + ".")
print("It contains " + str(pythondata.count(" ")) + " individual code tokens.")

# Preparing the dataset (or loading already processed dataset to not do everything again)
if (os.path.isfile('data/pythontraining_processed_' + mode)):
  with open ('data/pythontraining_processed_' + mode, 'rb') as fp:
    all_words = pickle.load(fp)
  print("loaded processed model.")
else:
  print("now processing...")
  processed = pythondata
  all_sentences = nltk.sent_tokenize(processed)
  all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
  print("saving")
  with open('data/pythontraining_processed_' + mode, 'wb') as fp:
    pickle.dump(all_words, fp)

print("processed.\n")

# for 반복문 시작 직전에 이 코드를 추가하여 all_words 변수를 확인합니다.

print("\n--- [진단 시작] all_words 변수 상태 확인 ---")
print(f"'all_words' 변수의 타입: {type(all_words)}")

if isinstance(all_words, list) and len(all_words) > 0:
    print(f"인식된 문장의 총 개수: {len(all_words)}")
    print(f"첫 번째 문장의 일부 (토큰 10개): {all_words[0][:10]}")
    print("진단 결과: 'all_words'가 정상적으로 보입니다.")
else:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!! [문제 발견] 'all_words' 변수가 비어있거나 리스트가 아닙니다. !!!")
    print("!!! Word2Vec 모델을 만들 재료가 없어 .npy 파일이 생성되지 않은 것 같습니다. !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

print("--- [진단 종료] ---\n")


# 원래 코드 시작
#trying out different parameters
for mincount in [10]:
  for iterationen in [6]:
    for s in [100]:

      print("\n\n" + mode + " W2V model with min count " + str(mincount) + " and " + str(iterationen) + " Iterationen and size " + str(s))
      fname = "w2v/word2vec_"+mode+str(mincount) + "-" + str(iterationen) +"-" + str(s)+ ".model"

      if (os.path.isfile(fname)):
        print("model already exists.")
        continue

      else:
        print("calculating model...")
        # training the model
        model = Word2Vec(all_words, vector_size=s, min_count=mincount, epochs=iterationen, workers = 4)
        #vocabulary = model.wv.key_to_index

        #print some examples

        #words = ["import", "true", "while", "if", "try", "in", "+", "x", "=", ":", "[", "print", "str", "count", "len", "where", "join", "split", "==", "raw_input"]
        #for similar in words:
        #  try:
        #    print("\n")
        #    print(similar)
        #    sim_words = model.wv.most_similar(similar)
        #    print(sim_words)
        #    print("\n")
        #  except Exception as e:
        #    print(e)
        #    print("\n")

        #saving the model
        model.save(fname)





Loading withString
Length of the training file: 337300692.
It contains 78328049 individual code tokens.
loaded processed model.
processed.


--- [진단 시작] all_words 변수 상태 확인 ---
'all_words' 변수의 타입: <class 'list'>
인식된 문장의 총 개수: 3769517
첫 번째 문장의 일부 (토큰 10개): ['__revision__', '=', '``', '$', 'id', '$', "''", 'distutils_all', '=', '[']
진단 결과: 'all_words'가 정상적으로 보입니다.
--- [진단 종료] ---



withString W2V model with min count 10 and 6 Iterationen and size 100
calculating model...


In [ ]:
from gensim.models import Word2Vec, KeyedVectors
import os

# 모델 파일 경로 (pSPigW-f_oke 셀에서 생성된 파일명 사용)
# 모델 훈련이 완료되지 않았다면 FileNotFoundError가 발생할 수 있습니다.
model_fname = "w2v/word2vec_withString10-6-100.model"
model_path = os.path.join(os.getcwd(), model_fname)


print(f"모델 파일 경로: {model_path}")

if os.path.exists(model_path):
    try:
        # Word2Vec 모델 로드
        model = Word2Vec.load(model_path)
        print("\n--- 모델 테스트 ---")

        # 테스트 단어 목록
        test_words = ["import", "true", "while", "if", "try", "in", "+", "=", "[", "print", "str", "count", "len", "split", "=="]

        for word in test_words:
            try:
                if word in model.wv:
                    print(f"\n'{word}'와(과) 유사한 단어:")
                    sim_words = model.wv.most_similar(word, topn=5)
                    for sim_word, score in sim_words:
                        print(f"  - {sim_word}: {score:.4f}")
                else:
                    print(f"'{word}'는(은) 모델의 어휘에 없습니다.")
            except Exception as e:
                print(f"'{word}' 테스트 중 오류 발생: {e}")

        print("\n--- 모델 테스트 완료 ---")

    except Exception as e:
        print(f"\n모델 로드 중 오류 발생: {e}")
else:
    print("\n[오류] 모델 파일을 찾을 수 없습니다.")
    print("Word2Vec 모델 훈련이 완료되었는지 확인해주세요.")

모델 파일 경로: /content/drive/MyDrive/fortmp/w2v/VulnerabilityDetection/Code/w2v/word2vec_withString10-6-100.model

--- 모델 테스트 ---

'import'와(과) 유사한 단어:
  - from: 0.5426
  - werkzeug: 0.5283
  - __all__: 0.5145
  - print_function: 0.5143
  - pyasn1: 0.4988

'true'와(과) 유사한 단어:
  - false: 0.9104
  - ,: 0.5298
  - none: 0.5112
  - ): 0.4936
  - :: 0.4891

'while'와(과) 유사한 단어:
  - break: 0.6164
  - -=: 0.4976
  - if: 0.4898
  - +=: 0.4760
  - line_newline_stripped: 0.4665

'if'와(과) 유사한 단어:
  - elif: 0.8061
  - assert: 0.6203
  - and: 0.5914
  - or: 0.5539
  - continue: 0.4978

'try'와(과) 유사한 단어:
  - else: 0.6119
  - finally: 0.5561
  - except: 0.4203
  - doesnotexist: 0.3936
  - check_mode: 0.3873

'in'와(과) 유사한 단어:
  - get_language_info: 0.5516
  - task=-1: 0.5255
  - purposes: 0.5166
  - incidental: 0.5126
  - sleeping: 0.4964

'+'와(과) 유사한 단어:
  - +=: 0.7072
  - -=: 0.5748
  - \'date: 0.5560
  - /: 0.5413
  - 0-9a-fa-f: 0.5380

'='와(과) 유사한 단어:
  - ,: 0.7453
  - ]: 0.6684
  - [: 0.6562
  - ): 0.6

In [ ]:
import os

# 1. 결과물이 생성된 폴더 경로
# 모델 파일은 w2v 폴더에 생성됩니다.
output_dir = "w2v"

# 2. 구글 드라이브에 최종 모델을 저장할 경로
drive_save_dir = "/content/drive/MyDrive/fortmp/w2v/trained_w2v_model"
os.makedirs(drive_save_dir, exist_ok=True)

# 3. 생성된 모델 파일 목록 확인
try:
    generated_files = os.listdir(output_dir)
    print(f"'{output_dir}' 폴더의 생성된 파일 목록:")
    for f in generated_files:
        print(f"- {f}")

    # 4. 생성된 모든 파일을 구글 드라이브로 복사
    # w2v 폴더 자체가 아닌 w2v 폴더 안의 내용물을 복사합니다.
    for item in os.listdir(output_dir):
        s = os.path.join(output_dir, item)
        d = os.path.join(drive_save_dir, item)
        if os.path.isdir(s):
            # 하위 폴더가 있다면 재귀적으로 복사
            !cp -r {s} {d}
        else:
            # 파일 복사
            !cp {s} {d}


    print(f"\n모델 파일들을 '{drive_save_dir}' 경로에 안전하게 저장했습니다!")

    # 5. 저장된 모델 테스트 (선택 사항)
    from gensim.models import Word2Vec

    # 드라이브에 저장된 모델 경로를 사용
    # 파일명은 실제 저장된 파일명과 일치해야 합니다. 위 ls 결과에서 확인하세요.
    model_filename = "word2vec_withString10-6-100.model" # 모델 훈련 설정에 따른 파일명
    model_path = os.path.join(drive_save_dir, model_filename)

    if os.path.exists(model_path):
        try:
            model = Word2Vec.load(model_path)
            print("\n--- 저장된 모델 로드 및 테스트 ---")
            # 테스트 단어는 필요에 따라 변경 가능
            test_word = "if"
            if test_word in model.wv:
                 print(f"'{test_word}'와 유사한 단어:", model.wv.most_similar(test_word, topn=5))
            else:
                 print(f"'{test_word}'는(은) 저장된 모델의 어휘에 없습니다.")

            test_word_2 = "split"
            if test_word_2 in model.wv:
                 print(f"'{test_word_2}'와 유사한 단어:", model.wv.most_similar(test_word_2, topn=5))
            else:
                 print(f"'{test_word_2}'는(은) 저장된 모델의 어휘에 없습니다.")

            print("\n--- 저장된 모델 테스트 완료 ---")

        except Exception as e:
            print(f"\n저장된 모델 로드 또는 테스트 중 오류 발생: {e}")
    else:
        print(f"\n[오류] 저장된 모델 파일을 찾을 수 없습니다. '{model_path}' 경로와 파일명을 확인해주세요.")

except FileNotFoundError:
    print(f"\n[오류] '{output_dir}' 폴더를 찾을 수 없습니다. 모델 파일이 생성되지 않았거나 경로가 잘못되었습니다.")
except Exception as e:
    print(f"\n파일 저장 또는 테스트 중 예상치 못한 오류 발생: {e}")

FileNotFoundError: [Errno 2] No such file or directory: 'models'

# 새 토크나이저 시험

In [ ]:
import re

def custom_tokenizer(code):
    # 코드를 소문자로 변환
    code = code.lower()

    # 조건문 패턴 정의 (예: if, while, for 뒤에 오는 조건 부분)
    # 이 부분은 더 정교하게 만들 필요가 있습니다.
    condition_pattern = re.compile(r'\b(if|while|for)\s+([^:]+):')

    # 식별자 패턴 정의 (알파벳으로 시작하고 알파벳, 숫자, 언더스코어가 이어지는 패턴)
    # 파이썬 키워드는 제외합니다.
    python_keywords = {'False', 'None', 'True', 'and', 'as', 'assert', 'async', 'await', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield'}
    identifier_pattern = re.compile(r'\b[a-z_][a-z0-9_]*\b')

    # 조건을 먼저 대체
    condition_count = 0
    def replace_condition(match):
        nonlocal condition_count
        condition_count += 1
        # 조건 부분을 ' condX' 형태로 대체하고, 원래 조건문 키워드와 콜론은 유지
        # 예: 'if not a:' -> 'if cond1:'
        return f'{match.group(1)} cond{condition_count}:'

    processed_code = condition_pattern.sub(replace_condition, code)


    # 식별자를 대체
    identifiers = {}
    identifier_count = 0

    def replace_identifier(match):
        nonlocal identifier_count
        word = match.group(0)
        if word in python_keywords or word.startswith('cond'): # 키워드나 이미 대체된 조건은 건너뛰기
            return word
        if word not in identifiers:
            identifier_count += 1
            identifiers[word] = f'a{identifier_count}'
        return identifiers[word]

    # 코드를 공백 기준으로 분할하여 식별자 대체 적용
    tokens = []
    # 조건을 대체한 코드에 대해 식별자 대체 적용
    for token in processed_code.split():
         # 각 토큰에 대해 식별자 패턴을 찾아 대체
         replaced_token = identifier_pattern.sub(replace_identifier, token)
         tokens.append(replaced_token)


    # 토큰들을 다시 문자열로 결합하거나 리스트 형태로 반환
    return " ".join(tokens) # 예시로 공백으로 다시 결합

# 테스트
code_snippet = """
def example_function(data):
    if not data:
        print("Data is empty")
    else:
        count = 0
        for item in data:
            if item > 10:
                count += item
        while count > 0:
            count -= 1
            print(f"Remaining: {count}")
    return count
"""

tokenized_code = custom_tokenizer(code_snippet)
print("원본 코드:")
print(code_snippet)
print("\n토큰화된 코드:")
print(tokenized_code)

원본 코드:

def example_function(data):
    if not data:
        print("Data is empty")
    else:
        count = 0
        for item in data:
            if item > 10:
                count += item
        while count > 0:
            count -= 1
            print(f"Remaining: {count}")
    return count


토큰화된 코드:
def a1(a2): if cond1: a3("a2 is a4") else: a5 = 0 for cond2: if cond3: a5 += a6 while cond4: a5 -= 1 a3(a7"a8: {a5}") return a5


In [ ]:
import nltk
from gensim.models import Word2Vec, KeyedVectors
import os.path
import pickle
import sys

# 기존의 custom_tokenizer 함수를 여기에 복사하거나,
# 이미 위의 셀에 정의되어 있다면 이 부분은 생략합니다.
# from your_tokenizer_file import custom_tokenizer # 만약 다른 파일에 저장했다면

all_words = []

mode = "withString_custom_tokenized" # 새로운 모드 이름

# Loading the training corpus
print("Loading " + mode)
# 원본 파일 로드
with open('w2v/pythontraining' + '_withString_X', 'r') as file:
    pythondata = file.read().lower().replace('\n', ' ')

print("Length of the training file: " + str(len(pythondata)) + ".")
print("It contains " + str(pythondata.count(" ")) + " individual code tokens before custom tokenization.")

# Preparing the dataset (or loading already processed dataset to not do everything again)
processed_data_path = 'data/pythontraining_processed_' + mode
if (os.path.isfile(processed_data_path)):
  with open (processed_data_path, 'rb') as fp:
    all_words = pickle.load(fp)
  print("loaded processed model.")
else:
  print("now processing with custom tokenizer...")
  # 새로운 토크나이저 적용
  tokenized_text = custom_tokenizer(pythondata)

  # 토큰화된 문자열을 띄어쓰기 기준으로 분할하여 리스트로 만듦
  # Word2Vec은 'list of lists' 형태의 입력이 필요하므로,
  # 여기서는 각 문장을 하나의 '문서'로 간주하고 전체를 하나의 리스트로 만듭니다.
  # 만약 문장 단위로 학습하고 싶다면, custom_tokenizer 함수를 수정하여 문장 단위로 토큰화하도록 해야 합니다.
  all_words = [tokenized_text.split()] # 전체를 하나의 문장(리스트)으로 처리

  print("saving processed data...")
  os.makedirs('data', exist_ok=True) # 'data' 폴더가 없으면 생성
  with open(processed_data_path, 'wb') as fp:
    pickle.dump(all_words, fp)

print("processed.\n")

# for 반복문 시작 직전에 이 코드를 추가하여 all_words 변수를 확인합니다.

print("\n--- [진단 시작] all_words 변수 상태 확인 ---")
print(f"'all_words' 변수의 타입: {type(all_words)}")

if isinstance(all_words, list) and len(all_words) > 0:
    print(f"인식된 문장의 총 개수 (여기서는 전체 파일이 1개의 문장): {len(all_words)}")
    if len(all_words[0]) > 10:
        print(f"첫 번째 문장의 일부 (토큰 10개): {all_words[0][:10]}")
    else:
         print(f"첫 번째 문장의 일부 (전체 토큰): {all_words[0]}")
    print("진단 결과: 'all_words'가 정상적으로 보입니다.")
else:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!! [문제 발견] 'all_words' 변수가 비어있거나 리스트가 아닙니다. !!!")
    print("!!! Word2Vec 모델을 만들 재료가 없어 학습이 어렵습니다. !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

print("--- [진단 종료] ---\n")


# trying out different parameters
for mincount in [10]:
  for iterationen in [6]:
    for s in [100]:

      print("\n\n" + mode + " W2V model with min count " + str(mincount) + " and " + str(iterationen) + " Iterationen and size " + str(s))
      fname = "w2v/word2vec_"+mode+"_" + str(mincount) + "-" + str(iterationen) +"-" + str(s)+ ".model"
      model_save_path = os.path.join(os.getcwd(), 'w2v', fname)


      if (os.path.isfile(model_save_path)):
        print("model already exists.")
        continue

      else:
        print("calculating model...")
        # training the model
        # Word2Vec 모델은 'list of lists' 형태를 기대합니다.
        # 우리의 all_words는 현재 [전체_토큰_리스트] 형태이므로, 그대로 사용합니다.
        model = Word2Vec(all_words, vector_size=s, min_count=mincount, epochs=iterationen, workers = 4)

        #saving the model
        model.save(model_save_path)
        print(f"Model saved to {model_save_path}")